# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [23]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from panel.interact import interact
from panel import widgets
from pathlib import Path
from dotenv import load_dotenv

In [24]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [25]:
# Import the necessary CSVs to Pandas DataFrames

file_path = Path("Data/mean_housing_units.csv")
mean_housing_units = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/mean_sale_price_per_sq_foot.csv")
mean_sale_price_per_sq_foot = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/mean_monthly_rent.csv")
mean_monthly_rent = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/mean_prices_for_year_and_neighborhood.csv")
mean_prices_for_year_and_neighborhood = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/top_10_expensive.csv")
top_10_expensive = pd.read_csv(file_path)

file_path = Path("Data/new_top_10_expensive_neighborhoods.csv")
new_top_10_expensive_neighborhoods = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/joined_sfo_data_with_coordinates.csv")
joined_sfo_data_with_coordinates = pd.read_csv(file_path)

file_path = Path("Data/df_expensive_neighborhoods_per_year.csv")
df_expensive_neighborhoods_per_year = pd.read_csv(file_path)


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [26]:
# Define Panel Visualization Functions
def get_housing_units_per_year():
    """Housing Units Per Year."""
    
    mean_housing_units_plot = px.bar(
        mean_housing_units,
        x=mean_housing_units.index,
        y='housing_units',
        title="Housing Units in San Francisco From 2010 to 2016", 
        labels={'housing_units':'Housing Units'},
        log_y=True
    )
    return mean_housing_units_plot



def get_average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    mean_monthly_rent_plot = px.line(
        mean_monthly_rent,
        x=mean_sale_price_per_sq_foot.index,
        y='gross_rent',
        title="Average Monthly Rent By Year", 
        labels={'gross_rent':'Monthly Rent'}
    )
    mean_monthly_rent_plot.update_traces(
        line_color='firebrick'
    )
    return mean_monthly_rent_plot


def get_average_sales_price():
    """Average Sales Price Per Year."""
    
    mean_sale_price_per_sq_foot_plot = px.line(
        mean_sale_price_per_sq_foot,
        x=mean_sale_price_per_sq_foot.index,
        y='sale_price_sqr_foot',
        title="Average Sale Price Per Square Foot By Year", 
        labels={'sale_price_sqr_foot':'Housing Units Price Per Square Foot'}
    )
    mean_sale_price_per_sq_foot_plot.update_traces(
        line_color='blue'
    )
    return mean_sale_price_per_sq_foot_plot


def get_average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    average_monthly_rent = mean_prices_for_year_and_neighborhood.hvplot(
        x="year",
        y="gross_rent",
        kind="line",
        ylabel="Average Monthly Rent",
        groupby="neighborhood",
        color="pink"
    )
    return average_monthly_rent


def get_top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    top_10_expensive_plot = px.bar(
        top_10_expensive,
        x='neighborhood',
        y='sale_price_sqr_foot',
        title="Top 10 Expensive Neighborhoods in San Francisco", 
        range_y=[0,1000],
        labels={'sale_price_sqr_foot':'Average Sale Price Per Square Foot'}
    )
    return top_10_expensive_plot

def get_most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    new_top_10_expensive_neighborhoods_plot = new_top_10_expensive_neighborhoods.hvplot.bar(
        x="year",
        y=["gross_rent", "sale_price_sqr_foot"],
        groupby="neighborhood",
        value_label="Num Housing Units",
        xlabel="Neighborhood",
        stacked=False,
        rot=70,
        width=800,
        height=400,
        title="SFO Comparison Of Average Price Per Square Foot VS Average Montly Rent By Year"
    )
    return new_top_10_expensive_neighborhoods_plot
    
    
def get_parallel_coordinates():
    """Parallel Coordinates Plot."""

    parallel_coordinates_plot = px.parallel_coordinates(
        top_10_expensive, 
        dimensions=[
            "sale_price_sqr_foot", 
            "housing_units",
            "gross_rent"
        ],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.diverging.Tealrose,
        height=400,
        width=1300,
        labels={
            'sale_price_sqr_foot':'Sale Price Per Square Foot', 
            'housing_units':'Housing Units',
            'gross_rent':'Gross Rent'
        }
    )
    return parallel_coordinates_plot


def get_parallel_categories():
    """Parallel Categories Plot."""
    
    parallel_categories_plot = px.parallel_categories(
        top_10_expensive, 
        dimensions=[
            "neighborhood", 
            "sale_price_sqr_foot", 
            "housing_units",
            "gross_rent"
        ],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.diverging.Tealrose,
        height=400,
        width=1300,
        labels={
            'neighborhood':'Neighborhood', 
            'sale_price_sqr_foot':'Sale Price Per Square Foot', 
            'housing_units':'Housing Units',
            'gross_rent':'Gross Rent'
        }
    )
    return parallel_categories_plot


def get_neighborhood_map():
    """Neighborhood Map."""

    neighborhood_scatter_mapbox = px.scatter_mapbox(
        joined_sfo_data_with_coordinates, 
        lat="Lat", 
        lon="Lon",     
        color="gross_rent", 
        size="sale_price_sqr_foot",
        title="Average Sale Price Per Square Foot and Gross Rent In San Francisco",
        color_continuous_scale=px.colors.sequential.Bluered, 
        hover_name="neighborhood",
        size_max=15,
        zoom=10,
        width=1000, 
        height=1000
    )
    return neighborhood_scatter_mapbox

def get_sunburst():
    """Sunburst Plot."""
    
    df_expensive_neighborhoods_per_year_sunburst = px.sunburst(
        df_expensive_neighborhoods_per_year, 
        path=[
            'year', 
            'neighborhood'
        ], 
        values='sale_price_sqr_foot',
        color='gross_rent', 
        title="Costs Analysis of Most Expensive Neighborhoods in San Francisco Per Year",
        color_continuous_scale=px.colors.sequential.Blackbody,
        width=800,
        height=800
    )
    return df_expensive_neighborhoods_per_year_sunburst
    

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [33]:
# Create The Dashboard

sfo_intro = pn.pane.Markdown(
    """
    #San Francisco - The Gem of The West.

    ##Let's show you why you should invest.

    ###Positively Improve Your Real Estate Portfolio!
  
    """
)

#Tabs with data 
tab_1 = pn.Column(
    "#San Francisco Rental Analysis - 2010 to 2016",
    get_housing_units_per_year(),
    get_average_gross_rent(),
    get_average_sales_price()
)

tab_2 = pn.Column(
    "# San Francisco Neighborhood Analysis - 2010 to 2016",
    get_average_price_by_neighborhood(),
    get_top_most_expensive_neighborhoods(),
    get_most_expensive_neighborhoods_rent_sales()
)

tab_3 = pn.Column(
    "# Parallel Coordinates and Parallel Categories Analysis",
    get_parallel_coordinates(),
    get_parallel_categories()
)

tab_4 = pn.Column(
    "# Average Sale Price Per Square Foot and Gross Rent in San Francisco",
    get_neighborhood_map(),
)

tab_5 = pn.Column(
    "# Sunburst Analysis",
    get_sunburst()
)


# Create dashboard Tabs
sfo_dashboard = pn.Tabs(
    ("San Francisco",sfo_intro),
    ("Rental Analysis",tab_1),
    ("Neighborhood Analysis", tab_2),
    ("Parallel Plots",tab_3),
    ("Map Analysis",tab_4),
    ("Sunburst Analysis", tab_5)
)

## Serve the Panel Dashboard

In [34]:
# Serve the dashboard
sfo_dashboard.servable()

Tabs
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure, visible=False)
        [2] Plotly(Figure, visible=False)
        [3] Plotly(Figure, visible=False)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] Plotly(Figure, visible=False)
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure, visible=False)
        [2] Plotly(Figure, visible=False)
    [4] Column
        [0] Markdown(str)
        [1] Plotly(Figure, visible=False)
    [5] Column
        [0] Markdown(str)
        [1] Plotly(Figure, visible=False)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [17]:
#get_housing_units_per_year()

In [8]:
#get_average_gross_rent()

In [9]:
# get_average_sales_price()

In [10]:
# get_average_price_by_neighborhood()

In [11]:
# get_top_most_expensive_neighborhoods()

In [12]:
# get_most_expensive_neighborhoods_rent_sales()

In [13]:
# get_neighborhood_map().show()

In [14]:
# get_parallel_categories()

In [15]:
# get_parallel_coordinates()

In [16]:
# get_sunburst()